In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing necessary libraries 

In [ ]:
import numpy as np
import pandas as pd

from warnings import filterwarnings
filterwarnings('ignore')

import gc
from scipy.stats import skew


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
import datatable as dt

# Making a funtion to reduce the memory usage as suggested by this [notebook](https://www.kaggle.com/lucamassaron/autogluon-for-tabular-playground-sep-2021).

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Dataset was processed as suggested in this [notebook](https://www.kaggle.com/realtimshady/single-simple-lightgbm) with standard scaler. If you want to try out new type of scalar , you can use this [dataset](https://www.kaggle.com/towhidultonmoy/tps-sep2021-dataset).It is imputed but not scaled.


In [ ]:
%%time
# read dataframe

X_test = pd.read_csv('../input/tps-sep21-standardscaled-dataset/TPS_Sep21_Test_StandardScaled.csv')
X_train = pd.read_csv('../input/tps-sep21-standardscaled-dataset/TPS_Sep21_Train_StandardScaled.csv')


sample_submission = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:


### REDUCE MEMORY USAGE
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)
gc.collect()



In [ ]:
X_test

# The scaled dataset had an extra column named 'Unnamed: 0' .As it is not necessary,I am just dropping it.

In [ ]:
X_train.drop(['Unnamed: 0'], axis=1, inplace=True)
X_test.drop(['Unnamed: 0'], axis=1, inplace=True)

# Where did I get this parameters? 
- I used optuna and also searched over kaggle to find the best parameters. I also had to change the parameters and see which parameters gave me the best results.

In [ ]:
seed=14


param = {
'objective': 'binary',
 #'n_estimators': 10000,
 'learning_rate': 0.05,
 'metric': 'auc',
 #'early_stopping_rounds': 1000,
 'reg_alpha': 4.731615831093071,
 'reg_lambda': 0.37831795775990107,
 'num_leaves': 8,
 'min_child_samples': 77,
 'max_depth': 5,
 'colsample_bytree': 0.19053128771270178,
 'min_data_per_group': 164
}
# param = {
#    #'n_estimators': 6630, 
#     'objective': 'binary',
#         'boosting': 'gbdt',
#         'metric' : 'auc',
#     'max_depth': 3, 
#     'learning_rate': 0.053625067203773684,
#     'reg_alpha': 4.618041066261469,
#     'reg_lambda': 7.9389723810790604,
#     'num_leaves': 203,
#     'min_data_per_group': 83, 
#     'min_child_samples': 141, 
#     'colsample_bytree': 0.13048987522123276   
# }
# params = {
#         'learning_rate': 0.13572437900113307,        
#         'lambda_l1': 2.154360665259325,
#         'lambda_l2': 6.711089761523827,
#         'num_leaves': 769,
#         '#min_sum_hessian_in_leaf': 20.44437160769411,
#         'feature_fraction': 0.7921473067441019,
#         'feature_fraction_bynode': 0.8083803860191322,
#         'bagging_fraction': 0.9726755660563261,
#         'bagging_freq': 42,
#         'min_data_in_leaf': 690,
#         'max_depth': 3,
#         'seed': seed,
#         'feature_fraction_seed': seed,
#         'bagging_seed': seed,
#         'drop_seed': seed,
#         'data_random_seed': seed,
#         'objective': 'binary',
#         'boosting': 'gbdt',
#         'metric' : 'auc',
#         'verbosity': -1,
#         'n_jobs': -1,
#     }  
# param = {'metric' : 'auc',
#     'max_depth' : 3,
#     'num_leaves' : 7,
#    # 'n_estimators' : 5000,
#     'colsample_bytree' : 0.3,
#     'subsample' : 0.5,
#     'random_state' : 7014,
#     'reg_alpha' : 18,
#     'reg_lambda' : 17,
#     'learning_rate' : 0.055,
#    # 'device' : 'gpu',
#     'objective' : 'binary'
#          }
# param = {'metric' : 'auc',
#     'max_depth' : 3,
#     'num_leaves' : 7,
#    # 'n_estimators' : 5000,
#     'colsample_bytree' : 0.3,
#     'subsample' : 0.5,
#     'random_state' : 7014,
#     'reg_alpha' : 18,
#     'reg_lambda' : 17,
#     'learning_rate' : 0.0055,
#    # 'device' : 'gpu',
#     'objective' : 'binary'
#          }


# param = {
#     'min_data_in_leaf': 106, 
#     'num_leaves': 500, 
#     'learning_rate': 0.08,
#     'min_child_weight': 0.03454472573214212,
#     'bagging_fraction': 0.4181193142567742, 
#     'feature_fraction': 0.3797454081646243,
#     'reg_lambda': 0.6485237330340494,
#     'reg_alpha': 0.3899927210061127,
#     'max_depth': -1, 
#     'objective': 'binary',
#     'seed': SEED,
#     'feature_fraction_seed': SEED,
#     'bagging_seed': SEED,
#     'drop_seed': SEED,
#     'data_random_seed': SEED,
#     'boosting_type': 'gbdt',
#     #'verbose': 1,
#     'metric':'auc',
#     'force_col_wise':True
#     }

# param ={
    
#         'num_leaves': 10,
#         'max_bin': 127,
#         'min_data_in_leaf': 11,
#         'learning_rate': 0.02,
#         'min_sum_hessian_in_leaf': 0.00245,
#         'bagging_fraction': 1.0, 
#         'bagging_freq': 5, 
#         'feature_fraction': 0.05,
#         'lambda_l1': 4.972,
#         'lambda_l2': 2.276,
#         'min_gain_to_split': 0.65,
#         'max_depth': 14,
#         'save_binary': True,
#         'seed': 1337,
#         'feature_fraction_seed': 1337,
#         'bagging_seed': 1337,
#         'drop_seed': 1337,
#         'data_random_seed': 1337,
#         'objective': 'binary',
#         'boosting_type': 'gbdt',
#         'verbose': 1,
#         'metric': 'auc',
#         'is_unbalance': True,
#         'boost_from_average': False,
# }

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
import lightgbm as lgb
from sklearn import metrics
import gc

In [ ]:
 X_train.columns

In [ ]:
X_train.claim.values

In [ ]:
%%time
nfold = 10

claim = 'claim'
predictors = X_train.loc[:, ~X_train.columns.isin(['claim','id'])].columns.values.tolist()

skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=14)
#skf = RepeatedKFold(n_splits=nfold,  n_repeats=2, random_state=14)
oof = np.zeros(len(X_train))
predictions = np.zeros(len(X_test))

i = 1
for train_index, valid_index in skf.split(X_train, X_train.claim.values):
    print("\nfold {}".format(i))
    xg_train = lgb.Dataset(X_train.iloc[train_index][predictors].values,
                           label=X_train.iloc[train_index][claim].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )
    xg_valid = lgb.Dataset(X_train.iloc[valid_index][predictors].values,
                           label=X_train.iloc[valid_index][claim].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )   

    
    clf = lgb.train(param, xg_train, 10000, valid_sets = [xg_valid], verbose_eval=50, early_stopping_rounds = 1000)
    oof[valid_index] = clf.predict(X_train.iloc[valid_index][predictors].values, num_iteration=clf.best_iteration) 
    
    predictions += clf.predict(X_test[predictors], num_iteration=clf.best_iteration) / nfold
    i = i + 1

print("\n\nCV AUC: {:<0.5f}".format(metrics.roc_auc_score(X_train.claim.values, oof)))

In [ ]:
test_df = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

**Here are some results, that I got changing some parameters. 
I got atleast0.15 increase of the Overall CV in the leaderboard.**

Overall CV: #0.81423 LB:0.81670

#for depth=3 #0.0.81434 LB: 0.81685

#lr=0.05 Overall CV:0.81450

#split=10 Overall CV:0.81581

#split=10 Overall CV:0.81632 LB:0.81809

#lr=0.06 Overall CV:0.81628


In [ ]:
test_df.head()

In [ ]:
sub_df = pd.DataFrame({"id": test_df.id.values})
sub_df["claim"] = predictions
sub_df[:10]

In [ ]:
sub_df.to_csv("lightgbm_cpu_new.csv", index=False)